In [1]:
import pandas as pd
import json
import requests
import math
import numpy as np
import urllib
import os

In [2]:
import utilcalcs as calc
import cen_geo_agg as geo

In [3]:
%load_ext dotenv
%dotenv
CensusAPI = os.environ.get('Census_API')

In [4]:
#My search parameters
year = '2018'
counties = ['005','047','061','081','085']
cols = f'B01001_001E,B01001_001M,group(B03002)' #add other tables here for other demo variables

source = 'acs/acs5'

In [5]:
def get_data(geo):
    frames = []
    for c in counties:
        if geo == 'cbg':
            url = f'https://api.census.gov/data/{year}/{source}?get={cols}&for=block%20group:*&in=state:36%20county:{c}&key={CensusAPI}'
            resp = requests.request('GET', url).content
            df = pd.DataFrame(json.loads(resp)[1:])
            df.columns = json.loads(resp)[0]
            frames.append(df)
        else:
            url = f'https://api.census.gov/data/{year}/{source}?get={cols}for=county:{c}&in=state:36&key={CensusAPI}'
            resp = requests.request('GET', url).content
            df = pd.DataFrame(json.loads(resp)[1:])
            df.columns = json.loads(resp)[0]
            frames.append(df)
    df = pd.concat(frames,sort=True)
    return df

def clean_data(df,var):
    dff = df[var].copy()
    var_num = var[1:]
    for col in var_num:
        dff[col] = dff[col].astype(float)
    dff = dff.replace([999999999, 555555555, 333333333, 222222222,\
                    666666666, 888888888, -999999999, -555555555,\
                    -333333333, -222222222, -666666666, -888888888], 0)
    return dff

In [6]:
## Total Population
Pop = ['B01001_001E','B01001_001M']

## Race 
WhiteAlone = ['B03002_003E','B03002_003M']
BlackAlone = ['B03002_004E','B03002_004M']
Hispanic = ['B03002_012E','B03002_012M']
AsianAlone = ['B03002_006E','B03002_006M']
OtherNonHisp_E = ['B03002_005E','B03002_007E','B03002_008E','B03002_009E'] #separate estimate & moe for calcs
OtherNonHisp_M = ['B03002_005M','B03002_007M','B03002_008M','B03002_009M']

Race = WhiteAlone + BlackAlone + Hispanic + AsianAlone + OtherNonHisp_E + OtherNonHisp_M

#other socioeconomic variables here
##
##
##


var_data = ['GEO_ID'] + Pop + Race # + other variables

In [7]:
df_cbg = get_data('cbg')

In [8]:
df_cbg = clean_data(df_cbg,var_data)

In [9]:
df_cbg.head(20)

,GEO_ID,B01001_001E,B01001_001M,B03002_003E,B03002_003M,B03002_004E,B03002_004M,B03002_012E,B03002_012M,B03002_006E,B03002_006M,B03002_005E,B03002_007E,B03002_008E,B03002_009E,B03002_005M,B03002_007M,B03002_008M,B03002_009M
0,1500000US360050245023,1886.0,358.0,11.0,23.0,227.0,134.0,1648.0,401.0,0.0,12.0,0.0,0.0,0.0,0.0,12.0,12.0,12.0,12.0
1,1500000US360050247002,1318.0,204.0,12.0,13.0,630.0,180.0,662.0,198.0,6.0,9.0,0.0,0.0,0.0,8.0,12.0,12.0,12.0,12.0
2,1500000US360050253004,1222.0,298.0,14.0,22.0,45.0,51.0,1155.0,299.0,0.0,12.0,0.0,0.0,0.0,8.0,12.0,12.0,12.0,13.0
3,1500000US360050263005,1941.0,610.0,464.0,291.0,398.0,254.0,891.0,504.0,0.0,12.0,126.0,0.0,27.0,35.0,115.0,12.0,35.0,41.0
4,1500000US360050265004,1191.0,623.0,0.0,12.0,211.0,295.0,892.0,529.0,88.0,147.0,0.0,0.0,0.0,0.0,12.0,12.0,12.0,12.0
5,1500000US360050266025,755.0,318.0,398.0,276.0,33.0,33.0,274.0,196.0,50.0,42.0,0.0,0.0,0.0,0.0,12.0,12.0,12.0,12.0
6,1500000US360050267011,1668.0,448.0,0.0,12.0,96.0,66.0,1056.0,419.0,463.0,236.0,0.0,0.0,0.0,53.0,12.0,12.0,12.0,83.0
7,1500000US360050213023,986.0,542.0,4.0,6.0,364.0,481.0,618.0,442.0,0.0,12.0,0.0,0.0,0.0,0.0,12.0,12.0,12.0,12.0
8,1500000US360050213022,863.0,526.0,0.0,12.0,426.0,460.0,424.0,220.0,13.0,21.0,0.0,0.0,0.0,0.0,12.0,12.0,12.0,12.0
9,1500000US360050213021,1709.0,836.0,0.0,12.0,705.0,657.0,976.0,613.0,28.0,58.0,0.0,0.0,0.0,0.0,12.0,12.0,12.0,12.0
